<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/Fine_Tuning_GPT2_using_custom_documents_04_10_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transfomers
!pip install -U PyPDF2
!pip install python-docx

ERROR: Could not find a version that satisfies the requirement transfomers (from versions: none)
ERROR: No matching distribution found for transfomers
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import re
from PyPDF2 import PdfReader
import docx
import os

In [4]:
def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

def read_word(file_path):
    doc = docx.Document(file_path)
    text = ''
    for paragraph in doc.paragraphs:
        text += paragraph.text + '\n'
    return text

def read_text_file(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    return text

def read_documents_From_directory(directory):
  combined_text = ""
  for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if filename.endswith(".pdf"):
      combined_text += read_pdf(file_path)
    elif filename.endswith(".docx"):
      combined_text += read_word(file_path)
    elif filename.endswith(".txt"):
      combined_text += read_text_file(file_path)
  return combined_text

In [6]:
train_directory = "/content"
text_data = read_documents_From_directory(train_directory)
text_data = re.sub(r"\n+", "\n", text_data).strip()

In [7]:
text_data

'### The Space King: Ghost Astronaut\nOnce upon a time, in a galaxy far away, there was a kingdom known as Starhaven. This magical kingdom was not like any ordinary realm; it floated among the stars, shimmering with colors no one had ever seen. The ruler of Starhaven was a kind and brave king named Astraeus, who had a deep love for the cosmos.\nKing Astraeus had a special gift: he could communicate with the stars and understand their secrets. He often ventured into the depths of space in his shining silver spaceship, the Celestial Voyager, to explore distant planets and meet extraordinary beings. His adventures were legendary, and he collected stories and treasures from every corner of the universe.\nBut one fateful day, during an expedition to the mysterious Nebula of Whispers, Astraeus encountered a dark force. A swirling storm appeared out of nowhere, and his ship was caught in its grip. Despite his bravery, Astraeus couldn’t escape. The storm swallowed the Celestial Voyager, and he

In [8]:
with open('/content/train.txt', 'w') as f:
    f.write(text_data)

In [10]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [11]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [12]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [13]:
def train(train_file_path, model_name, output_dir, overwrite_output_dir, per_device_train_batch_size, num_train_epochs, save_steps):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    data_collator = load_data_collator(tokenizer)
    train_dataset = load_dataset(train_file_path, tokenizer)

    tokenizer.save_pretrained(output_dir)

    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    trainer.train()
    trainer.save_model()

In [14]:
train_file_path = "/content/train.txt"
model_name = 'gpt2'
output_dir = '/content/result'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

In [15]:
train(train_file_path=train_file_path, model_name=model_name, output_dir=output_dir,
      overwrite_output_dir=overwrite_output_dir, per_device_train_batch_size=per_device_train_batch_size,
      num_train_epochs = num_train_epochs, save_steps = save_steps)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Step,Training Loss


# Inference

In [16]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [20]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model
def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path,  sequence, max_length):
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f"{sequence}", return_tensors = "pt")
    final_outputs = model.generate(
        ids,
        do_sample = True,
        max_length = max_length,
        pad_token_id = model.config.eos_token_id,
        top_k = 50,
        top_p = 0.95

    )
    print(tokenizer.decode(final_outputs[0],skip_special_tokens = True))

In [22]:
model1_path = "/content/result"
sequence1 = "[Q] What is the name of the kingdom mentioned here?"
max_length = 50
generate_text(model1_path, sequence1, max_length)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[Q] What is the name of the kingdom mentioned here? [A] The kingdom of Zhe-Rathir." [Q] Are these prophecies of divine providence still valid? [A] Indeed, the kingdom of God reign
